First thing to do is get the texts from Project Gutenberg.

Use method from this website to get text IDs:

https://skeptric.com/gutenberg/

Keep this package in mind for tokenization:

https://pypi.org/project/semchunk/

In [1]:
import csv
from collections import Counter
from io import BytesIO, StringIO
from pathlib import Path
import requests
import pandas as pd
# import gutenbergpy.textget
import json
from bs4 import BeautifulSoup
import re

In [2]:
# get catalog from Gutenberg
gut_cat = 'https://www.gutenberg.org/cache/epub/feeds/pg_catalog.csv.gz'

r = requests.get(gut_cat)
csv_text = r.content.decode('utf-8')

In [3]:
# get metadata for desired books
titles = ['Sense and Sensibility', 'Pride and Prejudice', 'Mansfield Park', 'Emma', 'Persuasion', 'Northanger Abbey']
author = 'Austen, Jane'

book_metadata = [book for book in csv.DictReader(StringIO(csv_text)) 
                 if author in book['Authors']
                 and book['Type'] == 'Text'
                 and book['Language'] == 'en'
                 and book['Title'] in titles]

In [4]:
# drop older versions of the text from metadata
old_versions = ['1342', '161']

book_metadata = [b for b in book_metadata if b['Text#'] not in old_versions]

In [5]:
id_lst = []

for book in book_metadata:
    id_lst.append(book['Text#'])

In [11]:
# regex pattern to remove header and footer
pattern = r'\*{3}'

# loop through to retrieve each in turn
for id in id_lst:
    print(id)
    url = f'https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt'
    res = requests.get(url)
    print(res.status_code)
    soup = BeautifulSoup(res.text)
    t = soup.find('p')
    raw = t.get_text()
    text = re.split(pattern, raw)[2]
    print(len(text))
    text = re.sub(r'[\r\n]+', ' ', text)
    final_text = re.sub(r' {2,}', ' ', text).strip()
    with open(f'../data/gut_books/{id}.txt', 'wt', encoding = 'utf-8') as book_file:
        book_file.write(final_text)
    

105
200
473207
121
200
442018
141
200
899252
158
200
896935
21839
200
703343
42671
200
705499


In [12]:
# save metadata
with open('../data/gut_books/austen_metadata.csv', 'wt') as f:
    csv_writer = csv.DictWriter(f, fieldnames = book_metadata[0].keys())
    csv_writer.writeheader()
    for b in book_metadata:
        csv_writer.writerow(b)